#### Helper stuff

In [ ]:
import numpy as np
from pathlib import Path
from collections import Counter
from scipy.stats import entropy
from scipy.spatial.distance import jensenshannon
from nlp_cyber_ner.dataset import read_iob2_file
from nlp_cyber_ner.dataset import read_aptner, read_cyner, remove_leakage
from nlp_cyber_ner.config import PROCESSED_DATA_DIR, RAW_DATA_DIR, INTERIM_DATA_DIR, TOKENPROCESSED_DATA_DIR
import spacy
from spacy.tokens import Doc



#### We will derive 3 matrices:
- dev to dev 
- train to train
- train (deduplicated) to dev

Interpretations:
- dev to dev: divergence between data that the model was evaluated - one from the same population as the training dataset, and another from a foreign one.
- train to train: divergence between the data that the model saw during training, vs the training data it would've (probably ideally - reason, 
I'm saying probably is because we may have a scenario where one model ends up performing better on a foreign dev set, than the model training
on the corresponding training set) liked to see during training. Relevant for comparison with model on the diagonal (a model that has the same train and dev).
- train (deduplicated) to dev - difference between the data the model was trained on, and the data the model was evaluated on. Some extra care for interpretation due to deduplication: The value in cell for row i, column j corresponds to the divergence between training set of dataset i, and the dev set of dataset j, where the training set has been deduplicated against the dev set. This is why, for instance, that the diagonal is not 0, necessarily.

#### Divergences for labels - token based counts

Potential problem here: The function is simply stripping prefixes and would count an entity of 3 different tokens with B-, I-, I- as 3 instances of that label - probably not good - should be fixed. I will create a version WITH and WITHOUT this fix.

In [2]:
def extract_entity_label(label):
    """
    If the label is in IOB2 format (e.g., "B-malware", "I-malware"),
    strip off the "B-" or "I-" prefix. Otherwise, return the label as is.
    """
    for prefix in ["B-", "I-"]:
        if label.startswith(prefix):
            return label[len(prefix):]
    return label

def get_label_distribution_ignoringprefix(data, label_set):
    """
    Given data (a list of sentences, where each sentence is a tuple of lists for words and labels, respectively),
    compute the distribution of the entity types specified in label_set.
    """
    counter = Counter()
    for sent in data:
        for label in sent[1]:
            # extract the entity label (removing IOB2 prefix, if any)
            ent_label = extract_entity_label(label)
            if ent_label in label_set:
                counter.update([ent_label])

    distribution = np.array([counter[l] for l in label_set], dtype=np.float64)
    # Add a tiny smoothing constant to avoid division by zero - probably not needed
    print(distribution)
    distribution += 1e-10
    distribution /= distribution.sum()
    return distribution

#### Divergence for labels - span based counts
A given span of labels for an entity only counts towards one occurance of that underlying entity category - i.e. concecutive B-MALWARE, I-MALWARE, I-MALWARE, would count as one MALWARE.

In [3]:
# This entire function can probably just be removed:

def extract_entity_labelandprefix(label):
    """
    If the label is in IOB2 format (e.g., "B-Malware", "I-Malware"),
    strip off the "B-" or "I-" prefix. Otherwise, return the label as is with "O"
    """
    for prefix in ["B-", "I-"]:
        if label.startswith(prefix):
            relevant_prefix = prefix
            return label[len(prefix):], relevant_prefix
    return label, "O"

def get_label_span_distribution(data, label_set):
    """
    Given data (a list of sentences, where each sentence is a tuple of lists for words and labels, respectively),
    compute the distribution of the entity types specified in label_set.
    """
    counter = Counter()
    for sent in data:
        previous_prefix = "O" #assuming entities don't span sentence boundaries
        for label in sent[1]:
            # extract the entity label (removing IOB2 prefix, if any)
            ent_label, prefix = extract_entity_labelandprefix(label)
            if (ent_label in label_set and prefix == "B-") or (ent_label in label_set and previous_prefix == "O"): 
                #the second case is if there happens to be an error in the annotation where the first prefix of the labeled span used prefix
                #I rather than B - feel like we caught some of that in our manual error analysis, but maybe I'm misremembering...
                counter.update([ent_label])
            previous_prefix = prefix

    distribution = np.array([counter[l] for l in label_set], dtype=np.float64)
    # Add a tiny smoothing constant to avoid division by zero - probably not needed
    distribution += 1e-10
    distribution /= distribution.sum()
    return distribution


#### Divergence for label distributions w. prefixes
Probably worth doing this since if datasets have vastly different lengths of their annotated entities, then this could highlight that? As an example, a malware entity that consists of 5 tokens would contribute a count of +1 to B-Malware and +4 to I-Malware

In [4]:
def get_label_distribution_wprefix(data):
    """
    Given data (a list of sentences, where each sentence is a tuple of lists for words and labels, respectively),
    compute the distribution of the entity types specified in label_set.
    """
    counter = Counter()
    for sent in data:
        for label in sent[1]:
            if label.startswith("B-") or label.startswith("I-"):
                counter.update([label])
    temp = np.sort(list(counter.keys()))
    distribution = np.array([counter[i] for i in temp], dtype=np.float64)
    # Add a tiny smoothing constant to avoid division by zero - probably not needed
    print(distribution)
    distribution += 1e-10
    distribution /= distribution.sum()
    return distribution

#### Divergence for word distributions

Currently the common vocabulary is just being constructed by considering all the datasets as one, and finding the 10000 most common words - there may be a better way of doing this - need to check the paper provided by Rob over email.

In [5]:
# Function to construct a common vocabulary across multiple datasets.
def get_global_vocab(datasets, max_vocab_size=None):
    """
    Given a list of datasets (each is a list of sentences, where each sentence 
    is a tuple (words, labels)), count the word frequencies and return the list
    of words from the most common to the less common.
    """
    global_counter = Counter()
    for data in datasets:
        for sent in data:
            # sent[0] contains the words
            global_counter.update(sent[0])
    if max_vocab_size is not None:
        vocab = [word for word, _ in global_counter.most_common(max_vocab_size)]
    else:
        vocab = list(global_counter.keys())
    return vocab

# Function to compute a word frequency distribution given a dataset and a fixed vocabulary.
def get_word_distribution(data, vocab):
    """
    Given data (a list of sentences, each sentence is a tuple (words, labels))
    and a fixed common vocabulary, compute a normalized frequency distribution
    of the words in 'vocab' for this dataset.
    """
    counter = Counter()
    for sent in data:
        counter.update(sent[0])
    # Create a vector in the same order as the vocabulary.
    distribution = np.array([counter[word] for word in vocab], dtype=np.float64)
    # Smoothing to avoid zero entries
    distribution += 1e-10
    # Normalize to form a probability distribution
    distribution /= distribution.sum()
    return distribution

#### Divergence for POS distribution
Using some off-the-shelf model for this to make predictions for the labels. Obviously we have no way of knowing the accuracy on the datasets so we're just gonna assume that the model is equally shit / good on each dataset - should be fine given we also have other metrics.

In [6]:
#https://huggingface.co/spacy/en_core_web_sm/tree/main
#https://spacy.io/api/attributeruler, https://github.com/explosion/spaCy/issues/5637
!python -m spacy download en_core_web_sm
!python -m spacy validate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 68.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.8.2) =================
ℹ spaCy installation:
/opt/anaconda3/envs/nlpproject_py312/lib/python3.12/site-packages/spacy

NAME             SPACY            VERSION                            
en_core_web_sm   >=3.8.0,<3.9.0   3.8.0   ✔



In [7]:
def get_pos_distributions(data):
    nlp = spacy.load("en_core_web_sm")
    #https://stackoverflow.com/a/71491525/24251578
    #I cant find the actual mapping from tags to more coarse POS tags that we're using here. 
    #Under the model's label scheme, you can only find the more fine-grained pos tags: https://spacy.io/models/en#en_core_web_sm.
    #There is a list of more coarse POS tags here, but they don't include the full output of this model: https://spacy.io/usage/linguistic-features#pos-tagging

    #Seems you can find it here but I ain't reading all that.
        #attr_ruler = nlp.get_pipe("attribute_ruler")
        #pprint.pprint(attr_ruler.patterns)
        #We'll just assume that each dataset ends up with the same set of labels used - which seems to be the case. The only reason this is 
        #important is because the distribution for JS needs to be the same dimension.
    counter = Counter()
    for sent in data:
        doc = Doc(nlp.vocab, sent[0])
        for token in nlp(doc):
            counter.update([token.pos_])
    temp = np.sort(list(counter.keys()))
    print(temp)
    distribution = np.array([counter[i] for i in temp], dtype=np.float64)
    # Add a tiny smoothing constant to avoid division by zero - probably not needed
    print(distribution)
    distribution += 1e-10
    distribution /= distribution.sum()
    return distribution

#### Create one for span length distributions

In [8]:
def get_spanlength_distribution(data):
    """
    Given data (a list of sentences, where each sentence is a tuple of lists for words and labels, respectively),
    compute the distribution of the labeled span lengths for entities in the dataset.
    """
    prefixes = ["B-", "I-"]
    counter = Counter({"1":0, "2":0, "3":0, "4":0, "5":0, "length>5":0})
    for sent in data:
        previous_prefix = "O"
        length = 0 #this is assuming that spans don't go across sentence boundaries - which must be true?
        for label in sent[1]:
            if (label.startswith("B-")) or (label.startswith("I-") and previous_prefix == "O"):
                for prefix in prefixes:
                    if label.startswith(prefix):
                        previous_prefix = prefix
                        break
                if length > 5:
                    counter["length>5"] += 1
                    length = 1
                    continue
                elif length >= 1:
                    counter[str(length)] += 1
                    length = 1
                    continue
            elif (label == "O" and previous_prefix in prefixes):
                if length > 5:
                    counter["length>5"] += 1
                    length = 0
                    continue
                elif length >= 1:
                    counter[str(length)] += 1
                    length = 0
                    continue              
            elif label.startswith("I-") and previous_prefix != "O":
                previous_prefix = "I-"
                length += 1   
    temp = np.sort(list(counter.keys()))
    distribution = np.array([counter[i] for i in temp], dtype=np.float64)
    # Add a tiny smoothing constant to avoid division by zero - probably not needed
    distribution += 1e-10
    distribution /= distribution.sum()
    return distribution


#### Compute JS and KL divergence

In [9]:
def compute_divergence_matrix(distributions: dict):
    dataset_names = list(distributions.keys())
    n_datasets = len(dataset_names)

    #matrices for kuhl and jensen
    js_matrix = np.zeros((n_datasets, n_datasets))
    kl_matrix = np.zeros((n_datasets, n_datasets))


    # For KL divergence, we compute the symmetric version: 0.5*(KL(P||Q) + KL(Q||P)). If not, the distance between two pairs of datasets may not
    # be the same depending on the 'perspective'
    for i in range(n_datasets):
        for j in range(n_datasets):
            P = distributions[dataset_names[i]]
            Q = distributions[dataset_names[j]]
            # Jensen-Shannon distance using SciPy (this is already symmetric)
            js_matrix[i, j] = jensenshannon(P, Q)
            # Symmetric KL divergence
            kl_sym = 0.5 * (entropy(P, Q) + entropy(Q, P))
            kl_matrix[i, j] = kl_sym
    return js_matrix, kl_matrix


#### Load data

In [11]:
dnrti_path = PROCESSED_DATA_DIR / "dnrti"
dnrti_train_path = dnrti_path / "train.unified"
dnrti_train_data = read_iob2_file(dnrti_train_path, word_index=0, tag_index=1)
dnrti_dev_path = dnrti_path / "valid.unified"
dnrti_dev_data = read_iob2_file(dnrti_dev_path)

attacker_path = PROCESSED_DATA_DIR / "attacker"
attacker_train_path  = attacker_path / "train.unified"
attacker_train_data = read_iob2_file(attacker_train_path, word_index=0, tag_index=1)
attacker_dev_path = attacker_path / "valid.unified"
attacker_dev_data = read_iob2_file(attacker_dev_path)

aptner_path = PROCESSED_DATA_DIR / "APTNer"
aptner_train_path = aptner_path / "train.unified"
aptner_train_data = read_iob2_file(aptner_train_path)
aptner_dev_path = aptner_path / "valid.unified"
aptner_dev_data = read_iob2_file(aptner_dev_path)

cyner_path = PROCESSED_DATA_DIR / "cyner"
cyner_train_path = cyner_path / "train.unified"
cyner_train_data = read_iob2_file(cyner_train_path)
cyner_dev_path = cyner_path / "valid.unified"
cyner_dev_data = read_iob2_file(cyner_dev_path)

#mismatched distribution dimensions?
#some of these functions don't have a predefined set of categories, but if it does happen that some datasets miss certain values, it'll cause
#errors during the JS calculations anyway - It can't be different dimensions, and it doens't seem to be the case that any of the datasets are 
#completely missing a certain category - can be fixed by just defining a fixed category set, but cba, since it all works.

#### Getting the numbers:

In [12]:
train_datasets = [
    ("DNRTI", dnrti_train_data),
    ("Attacker", attacker_train_data),
    ("APTNER", aptner_train_data),
    ("CyNER", cyner_train_data)
]

dev_datasets = [
    ("DNRTI", dnrti_dev_data),
    ("Attacker", attacker_dev_data),
    ("APTNER", aptner_dev_data),
    ("CyNER", cyner_dev_data)
]

LABEL_SET = ["Malware", "System", "Organization", "Vulnerability"]

all_datasets = [dnrti_train_data, attacker_train_data, aptner_train_data, cyner_train_data, dnrti_dev_data, attacker_dev_data, aptner_dev_data, cyner_dev_data]
common_vocab = get_global_vocab(all_datasets, max_vocab_size=10000)

distribution_methods = {
    "IgnoringPrefix":  lambda data: get_label_distribution_ignoringprefix(data, LABEL_SET),
    "SpanBased":       lambda data: get_label_span_distribution(data, LABEL_SET),
    "WithPrefix":      lambda data: get_label_distribution_wprefix(data),
    "WordDist":        lambda data: get_word_distribution(data, common_vocab),
    "POSDist":         lambda data: get_pos_distributions(data),
    "SpanLengthDist":  lambda data: get_spanlength_distribution(data)
}

for method_name, dist_func in distribution_methods.items():
    print("\n\n===========================================")
    print(f"=== DISTRIBUTION METHOD: {method_name} ===")
    print("===========================================")

    # -------------------------------------------
    # A) TRAIN-TO-TRAIN
    # -------------------------------------------
    train_distributions = {}
    for (name, data) in train_datasets:
        train_distributions[name] = dist_func(data)

    js_train_train, kl_train_train = compute_divergence_matrix(train_distributions)
    print("\n--- Train-to-Train ---")
    print("Jensen-Shannon Distance Matrix:")
    print(js_train_train)
    print("\nSymmetric KL Divergence Matrix:")
    print(kl_train_train)

    # -------------------------------------------
    # B) DEV-TO-DEV
    # -------------------------------------------
    dev_distributions = {}
    for (name, data) in dev_datasets:
        dev_distributions[name] = dist_func(data)

    js_dev_dev, kl_dev_dev = compute_divergence_matrix(dev_distributions)
    print("\n--- Dev-to-Dev ---")
    print("Jensen-Shannon Distance Matrix:")
    print(js_dev_dev)
    print("\nSymmetric KL Divergence Matrix:")
    print(kl_dev_dev)

    # -------------------------------------------
    # C) TRAIN(deduplicated)-TO-DEV
    # -------------------------------------------
    n = len(train_datasets)
    js_train_dev = np.zeros((n, n))
    kl_train_dev = np.zeros((n, n))

    for i in range(n):
        train_name, train_data = train_datasets[i]
        for j in range(n):
            dev_name, dev_data = dev_datasets[j]

            clean_data, _ = remove_leakage(train_data, dev_data)
            P = dist_func(clean_data)
            Q = dist_func(dev_data)

            js_train_dev[i, j] = jensenshannon(P, Q)
            kl_train_dev[i, j] = 0.5 * (entropy(P, Q) + entropy(Q, P))

    print("\n--- Train(deduplicated)-to-Dev ---")
    print("Jensen-Shannon Distance Matrix:")
    print(js_train_dev)
    print("\nSymmetric KL Divergence Matrix:")
    print(kl_train_dev)
    print("===========================================================")



=== DISTRIBUTION METHOD: IgnoringPrefix ===
[1777. 3835. 7937. 3219.]
[ 901. 1886. 2608.  914.]
[4252.  232. 6135.  516.]
[ 898. 1321.  400.   90.]

--- Train-to-Train ---
Jensen-Shannon Distance Matrix:
[[0.         0.07986706 0.34266991 0.36096433]
 [0.07986706 0.         0.34469139 0.28697856]
 [0.34266991 0.34469139 0.         0.44287795]
 [0.36096433 0.28697856 0.44287795 0.        ]]

Symmetric KL Divergence Matrix:
[[0.         0.02558817 0.53522315 0.55553726]
 [0.02558817 0.         0.56385104 0.34459174]
 [0.53522315 0.56385104 0.         1.00717131]
 [0.55553726 0.34459174 1.00717131 0.        ]]
[280. 473. 990. 388.]
[133. 301. 331. 151.]
[444.  81. 600.  19.]
[265. 273. 162.  14.]

--- Dev-to-Dev ---
Jensen-Shannon Distance Matrix:
[[0.         0.09389453 0.30797194 0.31487711]
 [0.09389453 0.         0.34131368 0.25890805]
 [0.30797194 0.34131368 0.         0.29883119]
 [0.31487711 0.25890805 0.29883119 0.        ]]

Symmetric KL Divergence Matrix:
[[0.         0.035432

In [16]:
all_results = {
    "IgnoringPrefix": {
        # updated raw label-count vectors (train splits)
        "LabelDistributions": {
            "DNRTI":     np.array([1777., 3835., 7937., 3219.]),
            "AttackNER": np.array([ 901., 1886., 2608.,  914.]),
            "APTNER":    np.array([4252.,  232., 6135.,  516.]),
            "CyNER":     np.array([ 898., 1321.,  400.,   90.])
        },
        "Train-to-Train_JS": np.array([
            [0.      , 0.07986706, 0.34266991, 0.36096433],
            [0.07986706, 0.      , 0.34469139, 0.28697856],
            [0.34266991, 0.34469139, 0.      , 0.44287795],
            [0.36096433, 0.28697856, 0.44287795, 0.      ]
        ]),
        "Train-to-Train_KL": np.array([
            [0.      , 0.02558817, 0.53522315, 0.55553726],
            [0.02558817, 0.      , 0.56385104, 0.34459174],
            [0.53522315, 0.56385104, 0.      , 1.00717131],
            [0.55553726, 0.34459174, 1.00717131, 0.      ]
        ]),
        "Dev-to-Dev_JS": np.array([
            [0.      , 0.09389453, 0.30797194, 0.31487711],
            [0.09389453, 0.      , 0.34131368, 0.25890805],
            [0.30797194, 0.34131368, 0.      , 0.29883119],
            [0.31487711, 0.25890805, 0.29883119, 0.      ]
        ]),
        "Dev-to-Dev_KL": np.array([
            [0.      , 0.03543253, 0.42711739, 0.43428761],
            [0.03543253, 0.      , 0.51776628, 0.29600349],
            [0.42711739, 0.51776628, 0.      , 0.3880076 ],
            [0.43428761, 0.29600349, 0.3880076 , 0.      ]
        ]),
        "TrainDedup-to-Dev_JS": np.array([
            [0.05354488, 0.10159504, 0.32967353, 0.33408855],
            [0.06914041, 0.04223393, 0.31592283, 0.26375104],
            [0.32756907, 0.36976507, 0.10593437, 0.3692673 ],
            [0.34529086, 0.27108238, 0.3819674 , 0.09414017]
        ]),
        "TrainDedup-to-Dev_KL": np.array([
            [0.01153706, 0.04145772, 0.49296368, 0.49289361],
            [0.01917479, 0.00714002, 0.43904301, 0.30049962],
            [0.48942613, 0.6540482 , 0.04741653, 0.68136262],
            [0.50537305, 0.30932209, 0.6512033 , 0.03565283]
        ])
    },
    "SpanBased": {
        "Train-to-Train_JS": np.array([
            [0.      , 0.13617656, 0.31196765, 0.3691458 ],
            [0.13617656, 0.      , 0.28220034, 0.27089175],
            [0.31196765, 0.28220034, 0.      , 0.42346006],
            [0.3691458 , 0.27089175, 0.42346006, 0.      ]
        ]),
        "Train-to-Train_KL": np.array([
            [0.      , 0.07614166, 0.43688426, 0.58493091],
            [0.07614166, 0.      , 0.37981549, 0.30479624],
            [0.43688426, 0.37981549, 0.      , 0.88910906],
            [0.58493091, 0.30479624, 0.88910906, 0.      ]
        ]),
        "Dev-to-Dev_JS": np.array([
            [0.      , 0.1177363 , 0.29543219, 0.36468549],
            [0.1177363 , 0.      , 0.25518505, 0.2925345 ],
            [0.29543219, 0.25518505, 0.      , 0.27911849],
            [0.36468549, 0.2925345 , 0.27911849, 0.      ]
        ]),
        "Dev-to-Dev_KL": np.array([
            [0.      , 0.05676031, 0.38686036, 0.5824755 ],
            [0.05676031, 0.      , 0.27376642, 0.35868638],
            [0.38686036, 0.27376642, 0.      , 0.3299068 ],
            [0.5824755 , 0.35868638, 0.3299068 , 0.      ]
        ]),
        "TrainDedup-to-Dev_JS": np.array([
            [0.04685495, 0.12868213, 0.3132396 , 0.38174753],
            [0.1224758 , 0.02629667, 0.23711523, 0.27958455],
            [0.30247096, 0.29710179, 0.13268362, 0.37969063],
            [0.35462154, 0.2792152 , 0.33235349, 0.08494395]
        ]),
        "TrainDedup-to-Dev_KL": np.array([
            [0.00881604, 0.06788617, 0.43802178, 0.64392931],
            [0.06149291, 0.00276825, 0.23520263, 0.32598835],
            [0.41167295, 0.42170814, 0.07445009, 0.68091531],
            [0.53662785, 0.32455711, 0.4794519 , 0.02894749]
        ])
    },
    "WithPrefix": {
        "LabelDistributions": {
            "DNRTI":     np.array([1221., 5529., 2449., 1896.,  556., 2408., 1386., 1323.]),
            "AttackNER": np.array([ 548., 1390.,  822.,  197.,  353., 1218., 1064.,  717.]),
            "APTNER":    np.array([3109., 4742.,  205.,  483., 1143., 1393.,   27.,   33.]),
            "CyNER":     np.array([ 703.,  284.,  837.,   48.,  195.,  116.,  484.,   42.])
        },
        "Train-to-Train_JS": np.array([
            [0.      , 0.17526216, 0.35662361, 0.36246624],
            [0.17526216, 0.      , 0.40123206, 0.31647466],
            [0.35662361, 0.40123206, 0.      , 0.4511981 ],
            [0.36246624, 0.31647466, 0.4511981 , 0.      ]
        ]),
        "Train-to-Train_KL": np.array([
            [0.      , 0.1271366 , 0.62326889, 0.56148289],
            [0.1271366 , 0.      , 0.87478338, 0.43151897],
            [0.62326889, 0.87478338, 0.      , 1.08575745],
            [0.56148289, 0.43151897, 1.08575745, 0.      ]
        ]),
        "Dev-to-Dev_JS": np.array([
            [0.      , 0.20736572, 0.33133543, 0.34510501],
            [0.20736572, 0.      , 0.41202332, 0.34592461],
            [0.33133543, 0.41202332, 0.      , 0.33482917],
            [0.34510501, 0.34592461, 0.33482917, 0.      ]
        ]),
        "Dev-to-Dev_KL": np.array([
            [0.      , 0.17871046, 0.56430059, 0.5244984 ],
            [0.17871046, 0.      , 1.06295504, 0.55254076],
            [0.56430059, 1.06295504, 0.      , 0.61245039],
            [0.5244984 , 0.55254076, 0.61245039, 0.      ]
        ]),
        "TrainDedup-to-Dev_JS": np.array([
            [0.05801686, 0.19746368, 0.35325099, 0.35786743],
            [0.18612717, 0.06471221, 0.38242566, 0.32570309],
            [0.33809908, 0.42432434, 0.13128673, 0.40985553],
            [0.34774526, 0.3154299 , 0.40147882, 0.15615116]
        ]),
        "TrainDedup-to-Dev_KL": np.array([
            [0.01354271, 0.16183411, 0.65248715, 0.57155545],
            [0.14337967, 0.01679723, 0.88743652, 0.47704894],
            [0.55443361, 1.01431812, 0.0723788 , 0.85200446],
            [0.51406981, 0.43269824, 0.91071757, 0.10223415]
        ])
    },
    "WordDist": {
        "Train-to-Train_JS": np.array([
            [0.      , 0.35760725, 0.13831401, 0.40274952],
            [0.35760725, 0.      , 0.3357986 , 0.37419193],
            [0.13831401, 0.3357986 , 0.      , 0.36833671],
            [0.40274952, 0.37419193, 0.36833671, 0.      ]
        ]),
        "Train-to-Train_KL": np.array([
            [0.      , 2.86754119, 0.32412469, 3.30223229],
            [2.86754119, 0.      , 2.46597506, 3.15551478],
            [0.32412469, 2.46597506, 0.      , 2.61344688],
            [3.30223229, 3.15551478, 2.61344688, 0.      ]
        ]),
        "Dev-to-Dev_JS": np.array([
            [0.      , 0.43331349, 0.37307906, 0.4707207 ],
            [0.43331349, 0.      , 0.40500397, 0.46921746],
            [0.37307906, 0.40500397, 0.      , 0.41378029],
            [0.4707207 , 0.46921746, 0.41378029, 0.      ]
        ]),
        "Dev-to-Dev_KL": np.array([
            [0.      , 5.28401586, 3.61465548, 5.94851187],
            [5.28401586, 0.      , 4.36613438, 6.16659823],
            [3.61465548, 4.36613438, 0.      , 4.47241725],
            [5.94851187, 6.16659823, 4.47241725, 0.      ]
        ]),
        "TrainDedup-to-Dev_JS": np.array([
            [0.31258369, 0.41659888, 0.32964615, 0.45355186],
            [0.38949561, 0.34087477, 0.34555944, 0.42380335],
            [0.31578788, 0.40334397, 0.29630735, 0.42663962],
            [0.43080244, 0.43586832, 0.36634973, 0.30713639]
        ]),
        "TrainDedup-to-Dev_KL": np.array([
            [2.37480618, 4.20415643, 2.41297445, 4.62927543],
            [3.90758582, 2.89750329, 2.97269921, 4.29816212],
            [2.29500266, 3.78459083, 1.91280747, 3.87598426],
            [4.6078584 , 4.854387  , 3.36104378, 2.31597525]
        ])
    },
    "POSDist": {
        "Train-to-Train_JS": np.array([
            [0.      , 0.04199404, 0.02111907, 0.05641533],
            [0.04199404, 0.      , 0.04111081, 0.0438426 ],
            [0.02111907, 0.04111081, 0.      , 0.04155997],
            [0.05641533, 0.0438426 , 0.04155997, 0.      ]
        ]),
        "Train-to-Train_KL": np.array([
            [0.      , 0.00728999, 0.00180301, 0.01295374],
            [0.00728999, 0.      , 0.0069864 , 0.00797699],
            [0.00180301, 0.0069864 , 0.      , 0.00692823],
            [0.01295374, 0.00797699, 0.00692823, 0.      ]
        ]),
        "Dev-to-Dev_JS": np.array([
            [0.      , 0.04413039, 0.04106488, 0.0607105 ],
            [0.04413039, 0.      , 0.04469234, 0.05426046],
            [0.04106488, 0.04469234, 0.      , 0.0312925 ],
            [0.0607105 , 0.05426046, 0.0312925 , 0.      ]
        ]),
        "Dev-to-Dev_KL": np.array([
            [0.      , 0.00919047, 0.00687539, 0.01489396],
            [0.00919047, 0.      , 0.00862162, 0.0125142 ],
            [0.00687539, 0.00862162, 0.      , 0.00392231],
            [0.01489396, 0.0125142 , 0.00392231, 0.      ]
        ]),
        "TrainDedup-to-Dev_JS": np.array([
            [0.01770765, 0.04133073, 0.04087191, 0.06165252],
            [0.04329741, 0.01661484, 0.04359336, 0.0485763 ],
            [0.02402037, 0.04209156, 0.02560897, 0.04625693],
            [0.05716238, 0.0485405 , 0.02797081, 0.02464628]
        ]),
        "TrainDedup-to-Dev_KL": np.array([
            [0.0012581 , 0.00746096, 0.00685219, 0.01541122],
            [0.00813797, 0.00110822, 0.0078107 , 0.00968479],
            [0.00231872, 0.00775144, 0.00263516, 0.00857853],
            [0.01322847, 0.01027207, 0.00313798, 0.00243258]
        ])
    },
    "SpanLengthDist": {
        "Train-to-Train_JS": np.array([
            [0.      , 0.22936515, 0.03539619, 0.05009848],
            [0.22936515, 0.      , 0.24052952, 0.19019921],
            [0.03539619, 0.24052952, 0.      , 0.06866643],
            [0.05009848, 0.19019921, 0.06866643, 0.      ]
        ]),
        "Train-to-Train_KL": np.array([
            [0.      , 0.26270405, 0.00509496, 0.01044159],
            [0.26270405, 0.      , 0.31248292, 0.16354544],
            [0.00509496, 0.31248292, 0.      , 0.01984823],
            [0.01044159, 0.16354544, 0.01984823, 0.      ]
        ]),
        "Dev-to-Dev_JS": np.array([
            [0.      , 0.28793086, 0.12249062, 0.13124202],
            [0.28793086, 0.      , 0.34799242, 0.21926447],
            [0.12249062, 0.34799242, 0.      , 0.15046813],
            [0.13124202, 0.21926447, 0.15046813, 0.      ]
        ]),
        "Dev-to-Dev_KL": np.array([
            [0.      , 0.88962542, 0.11451862, 0.46279672],
            [0.88962542, 0.      , 0.8020282 , 0.19822299],
            [0.11451862, 0.8020282 , 0.      , 0.2797514 ],
            [0.46279672, 0.19822299, 0.2797514 , 0.      ]
        ]),
        "TrainDedup-to-Dev_JS": np.array([
            [0.0754726 , 0.25898799, 0.14830318, 0.10556701],
            [0.26480789, 0.07800091, 0.31936959, 0.18108133],
            [0.08458899, 0.27316095, 0.13981647, 0.10533993],
            [0.10519283, 0.22426471, 0.17647083, 0.08704456]
        ]),
        "TrainDedup-to-Dev_KL": np.array([
            [0.16980204, 0.3483377 , 0.11222577, 0.05416863],
            [1.16205431, 0.02458686, 0.85702472, 0.13339517],
            [0.20248189, 0.43260172, 0.13043904, 0.06378875],
            [0.30332294, 0.23889539, 0.21647758, 0.0321578 ]
        ])
    }
}

In [17]:
from scipy.stats import pearsonr

#span-F1 performance matrix  (update if values change)
perf_matrix = np.array([
    [0.41, 0.16, 0.19, 0.07],
    [0.09, 0.23, 0.01, 0.02],
    [0.31, 0.16, 0.41, 0.18],
    [0.05, 0.04, 0.06, 0.40]
])

mask      = ~np.eye(len(dataset_names), dtype=bool)   # off-diagonal mask
perf_vec  = perf_matrix[mask]                         # 12-element vector

#correlate ONLY Jensen-Shannon matrices with performance
rows = []
for method, matdict in all_results.items():           # all_results already in memory
    for label, mat in matdict.items():
        if "JS" in label and isinstance(mat, np.ndarray) and mat.shape == (4, 4):
            div_vec      = mat[mask]
            r, p         = pearsonr(perf_vec, div_vec)
            rows.append({"Method": method,
                         "Matrix": label,
                         "Pearson_r": r,
                         "p_value": p})

corr_df = pd.DataFrame(rows).sort_values("Pearson_r")

#excel
out_file = "js_correlation.xlsx"
corr_df.to_excel(out_file, index=False, sheet_name="JS_Correlations")
print(f"Jensen–Shannon correlations written to {out_file}")


Jensen–Shannon correlations written to js_correlation.xlsx
